In [1]:
import numpy as np
import pandas as pd
import torch
import csv

In [24]:
# load model
H = torch.load("H_rank100.pt",map_location=torch.device('cpu'))
W = torch.load("W_rank100.pt",map_location=torch.device('cpu'))

In [33]:
# load book/music indices
music_ix = pd.read_csv("music_ix.csv", header=None)
books_ix = pd.read_csv("books_ix.csv", header=None)
music_meta = pd.read_csv("combinedMusicData_small.csv", header=None)[[0, 1, 5]]
#music_meta = music_meta[music_meta.iloc[:, 1] != 'misc']

In [34]:
music_meta.head()

,0,1,5
0,Can I Live,rap,spotify:track:20U1I32q6kX0Nmy4u8CvUg
1,Forgive Me Father,rap,spotify:track:1cvzmIBhdaQdC1DBbFnpWB
2,Down and Out,rap,spotify:track:5mEwhCsqadBZIE7SSrv1gC
3,Fly In,rap,spotify:track:6Nr3QjImPUhCpUgnykO7hm
4,Im Not You,rap,spotify:track:4h1xaUGnbHp867AZ4mBM3m


In [39]:
def createRow(idx):
    query_ix = idx
    
    song_scores = (W[query_ix] @ H.T)
    top_song_ix = np.argsort(song_scores.detach().numpy())[::-1][:1000]
    
    srt = np.sort(top_song_ix)+1
    
    music_df = music_meta.loc[srt]
    
    music_df.columns = ["title", 'tag', 'uri']
    music_df['score'] = song_scores.detach().numpy()[np.sort(top_song_ix)]
    
    music_df = music_df.sort_values('score',ascending=False)
    music_df = music_df[music_df.tag != 'misc']
    
    print("default")
    default_df = music_df[:15]
    print(default_df.head())

    for genre in music_df['tag'].value_counts().index.values:
        if genre != 'misc': # avoid misc category, they are not songs.
            print(genre, "----------------")
            playlist_df = music_df[music_df['tag'] == genre][:15]
            if len(playlist_df) >= 15 and genre:
                print(playlist_df.head())

In [40]:
createRow(0)

default
                      title   tag                                   uri  \
341066          Without You   pop  spotify:track:1tAutlhI9FwnKn7fVgmd5P   
290873            Skip Town   rap  spotify:track:4cQsV7J8n0qt6rkBBT1hPb   
439675  Bulletproof Diaries   rap  spotify:track:3zxvosFrPc6zZIOOEl4haC   
370854      Soldiers Things  rock  spotify:track:0ayEPUzuc10ICaevxKzfq0   
333839       Candy Flipping   rap  spotify:track:0IeKyuxpJmNc2KjfLtNtSw   

           score  
341066  0.008243  
290873  0.007707  
439675  0.007270  
370854  0.007203  
333839  0.007192  
rap ----------------
                      title  tag                                   uri  \
290873            Skip Town  rap  spotify:track:4cQsV7J8n0qt6rkBBT1hPb   
439675  Bulletproof Diaries  rap  spotify:track:3zxvosFrPc6zZIOOEl4haC   
333839       Candy Flipping  rap  spotify:track:0IeKyuxpJmNc2KjfLtNtSw   
278865                Uuhhh  rap  spotify:track:46QpllNMGgsXljvbeuXwN6   
9472                  Juice  rap  sp

In [ ]:
with open('matrix.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    # csvwriter.writerows(rows) # can write one row (lst) or multiple rows (lst of lsts)